In [20]:
import os
import ast
import pandas as pd
import numpy as np

In [21]:
papid_authid_dataset = pd.read_csv("data/authors_hepth_1991_2003.csv", dtype='str')
papid_authid_dataset.drop("Unnamed: 0", axis=1, inplace=True)

In [22]:
papid_authid_dataset.head()

,Author Id,Paper Id,Authors,Year
0,1,9112005,Christof Schmidhuber,1991
1,2,9112014,H. Lu,1991
2,3,9109003,Jadwiga Bienkowska,1991
3,4,9108020,A. Strominger,1991
4,5,9108020,J. A. Harvey,1991


In [23]:
pap_auth = {}
for key, row in papid_authid_dataset.iterrows():
    authId = row["Author Id"]
    papId = row["Paper Id"]
    if papId in pap_auth.keys():
        pap_auth[papId].append(authId)
    else:
        pap_auth[papId] = list(authId)

In [24]:
pap_auth_df = pd.DataFrame(pap_auth.items(), columns=["Paper Id", "Author Id"])
pap_auth_df.head()

,Paper Id,Author Id
0,9112005,[1]
1,9112014,"[2, 34, 30, 33]"
2,9109003,[3]
3,9108020,"[4, 5]"
4,9112074,"[6, 7, 11, 12]"


In [25]:
pap_auth_df.describe()

,Paper Id,Author Id
count,29624,29624
unique,29624,19578
top,0007043,"[2, 1]"
freq,1,56


In [26]:
pap_auth_df["Paper Id"] = pap_auth_df["Paper Id"].astype("str")
pap_auth_df.to_csv("papId_authId.csv", index=False)

In [34]:
pap_auth_df = pd.read_csv("papId_authId.csv", dtype='str')

In [28]:
single_author_word = pd.read_csv("single_author_topic_words.csv", dtype='str')
paper_topic_word = pd.read_csv("single_authored_papers_topic_model.csv", dtype='str')

In [29]:
single_author_word = single_author_word.set_index("Author Id").to_dict()["Topic Words"]
paper_topic_word = paper_topic_word.set_index("Paper Id").to_dict()["Topic Words"]

In [30]:
authors = single_author_word
len(authors)

4187

In [38]:
authors = single_author_word
papId_papWord = {}

for count in range(0, 4):
    
    for key, row in pap_auth_df.iterrows():
    
        rowAuth = ast.literal_eval(row["Author Id"])
        papId = row["Paper Id"]
        #print(type(papId))
        if papId not in paper_topic_word.keys():
            break
        papWords = paper_topic_word[papId]
    
        for i in rowAuth:
            
            if i in authors.keys():
                
                set1 = set(papWords)
                set2 = set(authors[i])
                papWords = set1 ^ set2
            
            else:
                check = 0
                aid  = 0
                for j in rowAuth:
                    if j not in authors.keys():
                        if check == 0:
                            aid = j
                            check += 1
                        else:
                            break
                authors[aid] = papWords
                
        papId_papWord[papId] = papWords  
                     
for key, row in pap_auth_df.iterrows():
    
    rowAuth = ast.literal_eval(row["Author Id"])
    papId = row["Paper Id"]
    
    if papId not in paper_topic_word.keys():
        break
    
    for i in rowAuth:
        if i not in authors.keys():
            authors[i] = papId_papWord[papId]   

In [39]:
len(authors)

4187

In [31]:
type(list(paper_topic_word.keys())[0])

str